# Build dictionary on Phoenix dataset

In [ ]:
import pandas as pd
df = pd.read_csv("/Users/liweijie/SLR/scripts/corpus/train.SI5.corpus.csv",sep='|')
df.head(100)

## Util functions

In [ ]:
import pandas as pd
import spacy
from collections import Counter
import time

def build_dictionary(files):
    start = time.time()
    lang_model = spacy.load('de_core_news_sm')
    end = time.time()
    print('load lang_model cost %.3f s'%(end-start))
    train = []
    # 合并annotation中的语料
    for file in files:
        df = pd.read_csv(file,sep='|')
        for i in range(len(df)):
            train.append(df.loc[i]['annotation'])

    # Create a dictionary which maps tokens to indices (train contains all the training sentences)
    freq_list = Counter()
    punctuation = ['_','NULL','ON','OFF','EMOTION','LEFTHAND','IX','PU']
    for sentence in train:
        sentence = [tok.text for tok in lang_model.tokenizer(sentence) if not tok.text in punctuation]
        freq_list.update(sentence)

    # 按照词的出现频率建立词典，词频越高索引越靠前
    freq_list = sorted(freq_list.items(),key=lambda item:item[1],reverse=True)
    dictionary = {}
    dictionary['<pad>'] = 0
    dictionary['<bos>'] = 1
    dictionary['<eos>'] = 2
    for i,item in enumerate(freq_list):
        dictionary[item[0]] = i+3
    print("Build dictionary successfully!")
    return dictionary

def process_sentence(sentence):
    punctuation = ['_','NULL','ON','OFF','EMOTION','LEFTHAND','IX','PU']
    sentence = [tok.text for tok in lang_model.tokenizer(sentence) 
        if not tok.text in punctuation]
    # sentence = ['<bos>'] +   sentence + ['<eos>']
    print(sentence)
    indices = [dictionary[word] for word in sentence 
        if word in dictionary.keys()]
    return indices

In [ ]:
train_file = "train.corpus.csv"
dev_file = "dev.corpus.csv"
dictionary = build_dictionary([train_file,dev_file])
lang_model = spacy.load('de_core_news_sm')

## Find out of vocabulary words

In [ ]:
train_file = '/Users/liweijie/SLR/scripts/corpus/train.SI5.corpus.csv'
train_dict = build_dictionary([train_file])
dev_file = '/Users/liweijie/SLR/scripts/corpus/dev.SI5.corpus.csv'
dev_dict = build_dictionary([dev_file])
test_file = '/Users/liweijie/SLR/scripts/corpus/test.SI5.corpus.csv'
test_dict = build_dictionary([test_file])
out_of_vocab = []
for k,v in dev_dict.items():
    if not k in train_dict.keys():
        out_of_vocab.append(k)
for k,v in test_dict.items():
    if not k in train_dict.keys() and not k in dev_dict.keys():
        out_of_vocab.append(k)
out_of_vocab

In [ ]:
train_file = '/Users/liweijie/SLR/scripts/corpus/train.corpus.csv'
train_dict = build_dictionary([train_file])
dev_file = '/Users/liweijie/SLR/scripts/corpus/dev.corpus.csv'
dev_dict = build_dictionary([dev_file])
out_of_vocab = []
for k,v in dev_dict.items():
    if not k in train_dict.keys():
        out_of_vocab.append(k)
out_of_vocab